# **Project: MapGAN**
## Authors: Neill Shikada and Melanie Sharif
## Professor: Robin Burke
## Course: INFO5604

If you want to run this code locally using Anaconda, follow the instructions [here](https://research.google.com/colaboratory/local-runtimes.html):
1. Open Anaconda command prompt

2. Ensure that Jupyter has the proper extensions enabled
  `jupyter-serverextension enable jupyter_http_over_ws`

3. Run Jupyter Notebook and allow access to Google Colab with this code
  `jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com'`
  
4. Click the dropdown in the top left of Google Colab Notebook, select `Connect to a local runtime`

5. Copy the `http://localhost:8888/?token=###`  link from the Anaconda command prompt

6. Paste into the Google Colab url space

7. It should connect! Then keep the Jupyter Notebook open the whole time you code as this is where your code is running locally

# Import Dependencies

1. Install any package that needs to be installed

In [ ]:
!pip install earthengine-api --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 244 kB 6.5 MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.334-py3-none-any.whl size=274127 sha256=d24271646671b53aab7c7e14522175dbd805afdca811778c075ae9d10a752622
  Stored in directory: /root/.cache/pip/wheels/0a/d9/6b/1601dd4443ffced1345b91e6635c334a0c21f492a180c1ad6a
Successfully built earthengine-api
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 0.1.332
    Uninstalling earthengine-api-0.1.332:
      Successfully uninstalled earthengine-api-0.1.332


In [ ]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2. Import all the necessary packages

In [18]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [20]:
import os

In [21]:
import numpy as np

In [22]:
# Folium will be used for visualization purposes. It is a package that enables Google-Maps-like interactive maps
import folium
print(folium.__version__)

0.12.1.post1


In [23]:
# Enable access to Google Earth Engine. If you are running this locally, you have to right click and open the link in a new tab (otherwise it'll come up with an error)
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=cICedt73rayGc0TX_zLGro9aPgdMZzFKyRb8BzHPJ4c&tc=pBVRRmVRaJV0MV7PMAVaEnhHvgYUo-F0AQj6CtBSdt4&cc=X_gy-ppdF9I586-EqTGwusINWYwRvzuIyEEZSRb5bG0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvuZqdo1cwMye6e3wJpQ-2-lvvO1Fvhtq7c-vcfTOdBcMj7OX-KdUNE

Successfully saved authorization token.


# Define Variables

## Specify Cloud Storage Bucket
Google Colab uses Google Storage Buckets to run any code, so you have to have one and specify it!

In [ ]:
OUTPUT_BUCKET = 'mapgan'

Define database variables

In [ ]:
L9SR = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
RGB_BANDS = ['B4', 'B3', 'B2']
BOULDER = [40.0150, -105.2705]

Define the file paths for data

In [ ]:
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension


In [24]:
# Your Earth Engine username.  This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'nesh5910@colorado.edu'

# Cloud Storage bucket into which training, testing and prediction 
# datasets will be written.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'mapgan'

# Use Landsat 8 surface reflectance data for predictors.
L9SR = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
SWISS = ee.ImageCollection("ORTHO/Switzerland/SWISSIMAGE/10cm")
# Use these bands for prediction.
# The bands are different for each dataset in Earth Engine, but we are looking for the raw RGB values of the images
SWISS_BANDS = ['R', 'G', 'B']
BANDS = ['B4', 'B3', 'B2']

BOULDER = [40.0150, -105.2705]
BERN = [46.9480, 7.4474]

# This is a trianing/testing dataset of points with known land cover labels.
LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')
# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'landcover'
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 3

# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo.TFRecord'
# Export imagery in this region.
EXPORT_REGION = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])
# The name of the Earth Engine asset to be created by importing
# the classified image from the TFRecord file in Cloud Storage.
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/Classified_pixel_demo'

In [25]:
# This function is from the sample Google Colab CNN code to import earth engine data into folium
# https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=7uSjLbh1LBeX

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
            print('Rendering Image')
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [34]:
Map = folium.Map(location=BERN, zoom_start=10)
image = (SWISS
        .filterBounds(ee.Geometry.Point(BERN[1], BERN[0]))
        .first())
Map.add_ee_layer(
    image, {'bands': ['R', 'G', 'B'], 'min': 1, 'max': 255}, 'image'
)
Map

Rendering Image


In [52]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
image_viz_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    # 'bands': ['SR_B1'],
    'min': 1,
    'max': 65455,
    # 'gamma': [0.95, 1.1, 1]
}

# Define a map centered on San Francisco Bay.
map_l8 = folium.Map(location=BOULDER, zoom_start=10)

# Add the image layer to the map and display it.
map_l8.add_ee_layer(L9SR, image_viz_params, 'false color composite')
display(map_l8)

# Build the Neural Network

[Google EE DNN example](https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb)

[Google EE CNN example](https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb)

In [ ]:
def build_gen_model():
  model = Sequential()

  model.add(Dense(7*7*128, input_dim=128))